In [1]:
#%% Importing modules and data
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import quandl
import seaborn as sns
import matplotlib.dates as dates
import matplotlib.ticker as ticker
from pandas_datareader import data
from sklearn.preprocessing import StandardScaler

%matplotlib inline

def save_xls(list_dfs, xls_path,sheet_names):
    writer = ExcelWriter(xls_path)
    for n, df in enumerate(list_dfs):
        df.to_excel(writer, sheet_names[n])
    writer.save()
    return

In [2]:
### Collecting data

spx = pd.read_csv('SPX.csv')
spx['Date'] = pd.to_datetime(spx['Date'])
spx = spx.set_index(['Date'])
spx = spx[['Close']]
spx.columns = ['spx']

spx['month'] = spx.index.month
spx['month_start'] = spx['month'].shift(1) - spx['month']
spx = spx[spx['month_start'] != 0]
del spx['month'], spx['month_start']

spx['Date'] = spx.index
for index, row in spx.iterrows():
    curr_year = index.year
    curr_month = index.month
    spx.ix[index, 1] = dt.datetime(curr_year, curr_month, 1)
    curr_month = index.month

spx['Date'] = pd.to_datetime(spx['Date'])
spx = spx.set_index(['Date'])

vxo = pd.read_csv('http://www.cboe.com/publish/scheduledtask/mktdata/datahouse/vixcurrent.csv',skiprows=1).dropna()
vxo['Date'] = pd.to_datetime(vxo['Date'])
vxo = vxo.set_index(['Date'])
vxo = vxo[['VIX Close']]
vxo.columns = ['vix']

vxo['month'] = vxo.index.month
vxo['month_start'] = vxo['month'].shift(1) - vxo['month']
vxo = vxo[vxo['month_start'] != 0]
del vxo['month'], vxo['month_start']

vxo['Date'] = vxo.index
for index, row in vxo.iterrows():
    curr_year = index.year
    curr_month = index.month
    vxo.ix[index, 1] = dt.datetime(curr_year, curr_month, 1)
    curr_month = index.month

vxo['Date'] = pd.to_datetime(vxo['Date'])
vxo = vxo.set_index(['Date'])

# Economic data from Quandl

#unemployment_rate = quandl.get("FRED/NROUST", authtoken="-ZxrLoLy3vKgLtLraaMn")
#total_fed_debt = quandl.get("FRED/GFDEBTN", authtoken="-ZxrLoLy3vKgLtLraaMn")
#fed_debt_public_held = quandl.get("FRED/FYGFDPUN", authtoken="-ZxrLoLy3vKgLtLraaMn")
#public_debt_to_gdp = quandl.get("FRED/GFDEGDQ188S", authtoken="-ZxrLoLy3vKgLtLraaMn")
#household_debt_to_gdp = quandl.get("FRED/HDTGPDUSQ163N", authtoken="-ZxrLoLy3vKgLtLraaMn")
#industrial_production_final = quandl.get("FRED/IPB50002NQ", authtoken="-ZxrLoLy3vKgLtLraaMn")


# Investing for a Living 6 macro factors
civ_unemployment_rate = quandl.get("FRED/UNRATE", authtoken="-ZxrLoLy3vKgLtLraaMn")
civ_unemployment_rate.columns = ['civ_unemployment_rate']

real_retail_and_services = quandl.get("FRED/RRSFS", authtoken="-ZxrLoLy3vKgLtLraaMn")
real_retail_and_services.columns = ['real_retail_and_services']

industrial_production_consumer_goods = quandl.get("FRED/IPDCONGD", authtoken="-ZxrLoLy3vKgLtLraaMn")
industrial_production_consumer_goods.columns = ['industrial_production_consumer_goods']

new_housing = quandl.get("FRED/PERMIT", authtoken="-ZxrLoLy3vKgLtLraaMn")
new_housing.columns = ['new_housing']

yield_spread = quandl.get("FRED/T10Y3MM", authtoken="-ZxrLoLy3vKgLtLraaMn")
yield_spread.columns = ['yield_spread']

ConfidenceIndex = quandl.get("DY2/M1030000138", authtoken="-ZxrLoLy3vKgLtLraaMn")
ConfidenceIndex.columns = ['ConfidenceIndex']

curr_month = 0
ConfidenceIndex['Date'] = ConfidenceIndex.index
for index, row in ConfidenceIndex.iterrows():
    curr_year = index.year
    curr_month = index.month
    ConfidenceIndex.ix[index, 1] = dt.datetime(curr_year, curr_month, 1)
    curr_month = index.month
ConfidenceIndex['Date'] = pd.to_datetime(ConfidenceIndex['Date'])
ConfidenceIndex = ConfidenceIndex.set_index(['Date'])

national_fin_conditions = quandl.get("FRED/NFCILEVERAGE", authtoken="-ZxrLoLy3vKgLtLraaMn")
national_fin_conditions.columns = ['national_fin_conditions']
national_fin_conditions['Date'] = 0

curr_month = 0
for index, row in national_fin_conditions.iterrows():
    if index.month == curr_month:
        national_fin_conditions.drop(index, inplace=True)
    else:
        curr_year = index.year
        curr_month = index.month
        national_fin_conditions.ix[index, 1] = dt.datetime(curr_year, curr_month, 1)
    curr_month = index.month
national_fin_conditions['Date'] = pd.to_datetime(national_fin_conditions['Date'])
national_fin_conditions = national_fin_conditions.set_index(['Date'])

df = pd.concat([spx, vxo, civ_unemployment_rate, real_retail_and_services,
               industrial_production_consumer_goods, new_housing,
               yield_spread, national_fin_conditions, ConfidenceIndex], axis = 1).dropna()

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:79: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:95: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.i

# Rough Guideline of Macro Indicators Flags

Unemployment Rate Rising > 4.6 <br>
National Financial Conditions (US Leverage) > 0.0 <br>
Yield Spread > 2 <br>

Industrial Production Consumer Goods Month Over Month Decline for 3 Consecutive Months <br>
Real Retail and Services Month Over Month Decline for 3 Consecutive Months <br>
New Housing Starts Month Over Month Decline for 3 Consecutive Months <br>

China Confidence Index Last Month Decline < -3%

In [3]:
display = df[['spx','vix','civ_unemployment_rate','yield_spread','national_fin_conditions']]
df_pct = df[['real_retail_and_services','industrial_production_consumer_goods',
             'new_housing','ConfidenceIndex']].pct_change()
display = pd.concat([display, df_pct], axis = 1)
display.tail()

spx    vix  civ_unemployment_rate  yield_spread  \
Date                                                                  
2017-06-01  2430.060059   9.89                    4.4          1.19   
2017-07-01  2429.010010  11.22                    4.3          1.23   
2017-08-01  2476.350098  10.09                    4.4          1.18   
2017-09-01  2476.550049  10.13                    4.2          1.15   
2017-10-01  2529.120117   9.45                    4.1          1.27   

            national_fin_conditions  real_retail_and_services  \
Date                                                            
2017-06-01                    -0.49                 -0.000344   
2017-07-01                    -0.50                  0.003710   
2017-08-01                    -0.51                 -0.004510   
2017-09-01                    -0.55                  0.013138   
2017-10-01                    -0.64                  0.001254   

            industrial_production_consumer_goods  new_housing  ConfidenceIndex  
Date                                                                            
2017-06-01                              0.001339     0.091610         0.011607  
2017-07-01                             -0.019619    -0.035294         0.011474  
2017-08-01                              0.008050     0.034146         0.000873  
2017-09-01                              0.019802    -0.036950         0.034002  
2017-10-01                              0.004794     0.074286         0.044688